# Smart Loans

Nowadays, individuals and small businesses take out loans for many reasons. Most studies were done on the bank side, with existing data, scientists build models and predict the risk associated with each loan, but rarely people stand on the borrower’s side and help them save the interests they don’t have to pay. When taking out loans, we usually just tell the bank or the lending company the amount of money we need, and they provided an interest rate plan that we can either leave or take. Most borrowers don’t understand the fanatical value behind the features of a loan such as term, and effect of the loan amount, and this could result user paying more interest than they need. Here I used the loan data over the past 8 years from the Leading Club (a peer to peer lending company), I will run regression, classification and feature engineering to understand the basic mechanism with interest rate assignment. By selecting the relevant features to understand how interest rates are assigned, I will develop a recommendation program that help borrower save on interest. On the Investor's side I would use machine learning to predict wather a loan will be paid off based on the loan features. This will be a good way to minimize the loss on the investor's side while.

In [1]:
import pandas as pd
import numpy as np
import os
import imageio
import datetime
import time
import sqlite3
import operator
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, accuracy_score, recall_score, f1_score, roc_auc_score
from sklearn.metrics import roc_curve, auc
from datetime import timedelta, datetime

In [2]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.metrics import make_scorer, roc_auc_score, accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
import seaborn as sns
from scipy import stats
import math
% matplotlib inline
# Always make it pretty.
plt.style.use('ggplot')

## Data Exploration 

In [3]:
loan = pd.read_csv('loan.csv')
loanT = pd.DataFrame.copy(loan)

/Volumes/Data/Anaconda/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (19,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
def Data_clean(loan):
    loan_useless = ['id', 'member_id','url', 'desc', 'verification_status']
    loan.drop(loan_useless,axis=1,inplace = True)
    loan['earliest_cr_line'] = pd.to_datetime(loan['earliest_cr_line'])
    loan['issue_d'] = pd.to_datetime(loan['issue_d'])
    loan['last_pymnt_d'] = pd.to_datetime(loan['last_pymnt_d'])
    loan['next_pymnt_d'] = pd.to_datetime(loan['next_pymnt_d'])
    loan['last_credit_pull_d'] = pd.to_datetime(loan['last_credit_pull_d'])
    loan['dti_new'] = np.where(loan['application_type']=='JOINT', loan['dti_joint'],loan['dti'])
    loan['annual_inc'] = np.where(loan['application_type']=='JOINT', loan['annual_inc_joint'],loan['annual_inc'])
    loan_col = loan.columns.values
    loan_na = loan.isnull().sum()/len(loan)
    loan['log_annual_inc'] = loan['annual_inc'].apply([lambda x: math.log(x)])
    loan['log_int_rate']= loan['int_rate'].apply([lambda x: math.log(x)])
    loan['log_tot_cur_bal']= loan['tot_cur_bal'].apply([lambda x: math.log(x+1)])
    loan.loc[loan['home_ownership'] =='ANY','home_ownership']  ='OTHER'
    loan.loc[loan['home_ownership'] =='NONE','home_ownership']  ='OTHER'
    from datetime import timedelta, datetime
    today = datetime.now()
    loan['cre_len']=loan['earliest_cr_line'].apply(lambda x: (today - x).days)
    for i in range(len(loan_na)):
        if loan_na[i]>=0.8:
            #print(loan_na[i], loan_col[i],i)
            loan.drop(loan_col[i], axis = 1, inplace = True)

    for i in loan.columns.values:
        #print(i)
        if loan[i].dtypes == object or loan[i].dtypes == 'datetime64[ns]':
            loan_group = loan.groupby('term').size()
            maj = loan_group[loan_group == loan_group.max()].index[0]
            loan[i].fillna(maj, inplace = True);
        else:
            loan[i].fillna(loan[i].mean(skipna = True), inplace = True);
    return loan

In [18]:
def int_rate_plot(int_rate):
    
    #The distribution of the int_rate looks like the following, it rages from 4 % to 30%
    ax = sns.kdeplot(loan['int_rate'])
    ax.set(xlabel = 'int_rate percentage', ylabel = 'density')
    plt.show()

In [19]:
def corr_heatmap(loan):
    #This function plot the correlation matrix as a heat map
    corr = loan.corr()
    sns.heatmap(corr, 
            xticklabels=corr.columns.values,
            yticklabels=corr.columns.values)

In [20]:
def term_t_test(loan):
    #The function run the t test for term, take the data frame as input and return the pvalue
    int_36 = loan.loc[loan['term']==' 36 months', 'int_rate']
    int_60 = loan.loc[loan['term']==' 60 months', 'int_rate']
    t_test = stats.ttest_ind(int_36,int_60)
    term_plot = sns.boxplot(x='term',y='int_rate',data = loan)
    return t_test.pvalue

In [21]:
def list_stutas_t_test(loan):
    int_f = loan.loc[loan['initial_list_status']=='f', 'int_rate']
    int_w = loan.loc[loan['initial_list_status']=='w', 'int_rate']
    t_test = stats.ttest_ind(int_f,int_w)
    initial_list_plot = sns.boxplot(x='initial_list_status',y='int_rate',data = loan)
    return t_test.pvalue

In [22]:
def plot_fun_facts(loan):
    purpose_plot = sns.boxplot(x='purpose', y='int_rate', data = loan)

In [5]:
Data_clean(loan)

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,...,application_type,acc_now_delinq,tot_coll_amt,tot_cur_bal,total_rev_hi_lim,dti_new,log_annual_inc,log_int_rate,log_tot_cur_bal,cre_len
0,5000.0,5000.0,4975.000000,36 months,10.65,162.87,B,B2,36 months,10+ years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,27.65,10.085809,2.365560,11.214634,12137.0
1,2500.0,2500.0,2500.000000,60 months,15.27,59.83,C,C4,Ryder,< 1 year,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,1.00,10.308953,2.725890,11.214634,6934.0
2,2400.0,2400.0,2400.000000,36 months,15.96,84.33,C,C5,36 months,10+ years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,8.72,9.413444,2.770086,11.214634,5989.0
3,10000.0,10000.0,10000.000000,36 months,13.49,339.31,C,C1,AIR RESOURCES BOARD,10+ years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,20.00,10.803649,2.601949,11.214634,8089.0
4,3000.0,3000.0,3000.000000,60 months,12.69,67.79,B,B5,University Medical Group,1 year,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,17.94,11.289782,2.540814,11.214634,8120.0
5,5000.0,5000.0,5000.000000,36 months,7.90,156.46,A,A4,Veolia Transportaton,3 years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,11.20,10.491274,2.066863,11.214634,4893.0
6,7000.0,7000.0,7000.000000,60 months,15.96,170.08,C,C5,Southern Star Photography,8 years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,23.51,10.757988,2.770086,11.214634,4651.0
7,3000.0,3000.0,3000.000000,36 months,18.64,109.43,E,E1,MKC Accounting,9 years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,5.35,10.778956,2.925310,11.214634,4102.0
8,5600.0,5600.0,5600.000000,60 months,21.28,152.39,F,F2,36 months,4 years,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,5.55,10.596635,3.057768,11.214634,5107.0
9,5375.0,5375.0,5350.000000,60 months,12.69,121.45,B,B5,Starbucks,< 1 year,...,INDIVIDUAL,0.0,225.70261,139458.189336,32068.620045,18.08,9.615805,2.540814,11.214634,4954.0


In [4]:
loan_col = loan.columns.values

In [5]:
loan.shape

(887379, 74)

There are many missing values in this data, I will do some feature engineering to clean up the data set. (Inpute missing data, remove useless column, remove colunms with more than 80% data missing

In [6]:
loan_useless = ['id', 'member_id','url', 'desc', 'verification_status']
loan.drop(loan_useless,axis=1,inplace = True)

In [7]:
loan['earliest_cr_line'] = pd.to_datetime(loan['earliest_cr_line'])
loan['issue_d'] = pd.to_datetime(loan['issue_d'])
loan['last_pymnt_d'] = pd.to_datetime(loan['last_pymnt_d'])
loan['next_pymnt_d'] = pd.to_datetime(loan['next_pymnt_d'])
loan['last_credit_pull_d'] = pd.to_datetime(loan['last_credit_pull_d'])
loan['dti_new'] = np.where(loan['application_type']=='JOINT', loan['dti_joint'],loan['dti'])
loan['annual_inc'] = np.where(loan['application_type']=='JOINT', loan['annual_inc_joint'],loan['annual_inc'])

In [8]:
loan_col = loan.columns.values
loan_na = loan.isnull().sum()/len(loan)
for i in range(len(loan_na)):
    if loan_na[i]>=0.8:
        #print(loan_na[i], loan_col[i],i)
        loan.drop(loan_col[i], axis = 1, inplace = True)

In [9]:
for i in loan.columns.values:
    #print(i)
    if loan[i].dtypes == object or loan[i].dtypes == 'datetime64[ns]':
        loan_group = loan.groupby('term').size()
        maj = loan_group[loan_group == loan_group.max()].index[0]
        loan[i].fillna(maj, inplace = True)
    else:
        loan[i].fillna(loan[i].mean(skipna = True), inplace = True);

In [8]:
'a' < 'b'

True

In [9]:
import sys

if __name__ == "__main__":
    s = input().strip()

thsifjtf


In [13]:
s.split


ValueError: empty separator

In [19]:
s_list = list(s)
s_set = set(s)
Ans = [[i, s_list.count(i)] for i in s_set]
print(Ans.sort(key=lambda x: x[1],reverse=True))


In [15]:
sorted(set([i for i in mylist if mylist.count(i)>2]))

AttributeError: 'str' object has no attribute 'value_count'

In [24]:
Ans.sort(key=lambda x: x[1],reverse=True)

In [25]:
Ans

[['f', 2], ['t', 2], ['h', 1], ['i', 1], ['j', 1], ['s', 1]]